In [1]:
!pip install gliner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.3/76.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 61.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 66.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 65.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.3 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.22.0
    Uninstalling tokenizers-0.22.0:
      Successfully uninstalled tokenizers-0.22.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.56.1
    Uninstalling transformers-4.56.1:
      Successfully uninstalled transformers-4.56.1


In [2]:
from gliner import GLiNER

model = GLiNER.from_pretrained("urchade/gliner_multi")



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

pytorch_model.bin:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

gliner_config.json:   0%|          | 0.00/734 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:559: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Cristiano Ronaldo dos Santos Aveiro => person
5 February 1985 => date
Saudi Pro League => competitions
Al Nassr => teams
Portugal national team => teams
Ballon d'Or => award
UEFA Men's Player of the Year Awards => award
European Golden Shoes => award
UEFA Champions Leagues => competitions
UEFA European Championship => competitions
UEFA Nations League => competitions
Champions League => competitions
European Championship => competitions


In [109]:

text = """
11:49:05 I'll revert regarding BANK ABC to try to do another 200 mio at 2Y
FR001400QV82 AVMAFC FLOAT 06/30/28
offer 2Y EVG estr+45bps
estr average Estr average / Quarterly interest payment
"""

labels = [
    "bank name", "transaction amount", "isin code", "fr-id",
    "security description", "mature date", "offer term", "Evg",
    "estr_unit_small", "payment_duration_quaterly","date",'+bps','offer'
]
entities = model.predict_entities(text, labels, threshold=0.25)
for entity in entities:
    print(f"{entity['label']:<15} => {entity['text']}")

date            => 11:49:05
bank name       => BANK ABC
transaction amount => 200 mio
date            => 06/30/28
+bps            => 45bps
estr_unit_small => estr
payment_duration_quaterly => Quarterly


In [35]:
model2 = GLiNER.from_pretrained("urchade/gliner_large-v2.1")


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

gliner_config.json:   0%|          | 0.00/477 [00:00<?, ?B/s]

.gitattributes: 0.00B [00:00, ?B/s]

README.md: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.78G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/580 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:559: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [110]:
import json
from collections import defaultdict


entities = model2.predict_entities(text, labels, threshold=0.5)
refined_output = defaultdict(list)
for entity in entities:
    label = entity['label'].strip()
    refined_output[label].append(entity['text'])
refined_output = dict(refined_output)

print(json.dumps(refined_output, indent=4))
offer=refined_output.get("bank offer", ["N/A"])[0]
counterparty = refined_output.get("bank name", ["N/A"])[0]
notional = refined_output.get("transaction amount", ["N/A"])[0]
isin = refined_output.get("fr-id", ["N/A"])[0]
underlying = f"{refined_output.get('isin code', [''])[0]} FLOAT {refined_output.get('mature date', [''])[0]}"
maturity = f"{refined_output.get('offer term', [''])[0]} {refined_output.get('Evg', ['', ''])[0]}"
bid = f"{refined_output.get('estr_unit_small', [''])[0]}{refined_output.get('+bps', [''])[0]}"
payment_full = refined_output.get("payment_duration_quaterly", [""])[0]
payment_frequency = payment_full.split()[0] if payment_full else "N/A"


print(f"Counterparty:{counterparty} \nNotional:{notional} \nISIN:{isin} \nUnderlying:{underlying} \nbid {bid} \nOffer: {offer} \nPaymentFrequency:{payment_frequency}")

{
    "bank name": [
        "BANK ABC"
    ],
    "transaction amount": [
        "200 mio"
    ],
    "offer term": [
        "2Y",
        "2Y"
    ],
    "fr-id": [
        "FR001400QV82"
    ],
    "isin code": [
        "AVMAFC"
    ],
    "mature date": [
        "06/30/28"
    ],
    "offer": [
        "offer"
    ],
    "Evg": [
        "EVG"
    ],
    "estr_unit_small": [
        "estr",
        "estr",
        "Estr average"
    ],
    "+bps": [
        "+45bps"
    ],
    "payment_duration_quaterly": [
        "Quarterly interest payment"
    ]
}
Counterparty:BANK ABC 
Notional:200 mio 
ISIN:FR001400QV82 
Underlying:AVMAFC FLOAT 06/30/28 
bid estr+45bps 
Offer: N/A 
PaymentFrequency:Quarterly
